In [3]:
# Import modules
import tensorflow as tf
import keras as K
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import tensorflow_hub as hub
from keras.models import load_model

import pandas as pd
import numpy as np
import PIL
import logging
import os

import zipfile
import shutil

from datetime import datetime

# for figures
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns

# for the Grad-CAMs
from IPython.display import Image, display
import matplotlib.cm as cm

# Working Directory on A4 Computer (GoPro and V2 data are on the A4 Computer)
files = os.listdir()

os.chdir("/home/scott/Dropbox/IMAGINE Project/MSSI_Project/")
cwd = os.getcwd()
print("Current working directory:", cwd)

# This code allows GPU memory allocation to grow as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2024-06-12 15:38:05.682730: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 15:38:05.682767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 15:38:05.682808: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 15:38:05.692412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 15:38:06.811285: W tensorflow/compiler/

Current working directory: /home/scott/Dropbox/IMAGINE Project/MSSI_Project


In [4]:
#  Prints the Tensorflow, Keras versions, and number of GPUs
print(tf.__version__)
print(K.__version__)
# print("TensorFlow Hub version:", hub.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.14.0
2.14.0
Num GPUs Available:  4


In [5]:
from platform import python_version

print(python_version())

3.9.19


In [6]:
!nvidia-smi

Wed Jun 12 15:38:18 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     On   | 00000000:05:00.0  On |                  N/A |
| 23%   37C    P8    11W / 250W |    250MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN Xp     On   | 00000000:06:00.0 Off |                  N/A |
| 23%   

In [7]:
#This compiled file incorporate all existing data with GoPro in training set only

metadata = pd.read_csv("compiled data/metadata_goproUFP_V2_V3_04252024.csv", low_memory=False)
# Train: 100% GoPro + 80% Imagine V2 and V3
# Val: 10% Imagine V2 and V3
# Tst: 10% Imagine V2 and V3

In [18]:
# Check for NaNs in the entire DataFrame
# print(metadata.isna().sum())

In [16]:
# Ensure no NaNs in data
# assert not np.any(np.isnan(metadata['ln_ufp_num_10s_ma_image_label_raw']))
# assert not np.any(np.isnan(metadata[image_path]))

In [19]:
# print(metadata['image_path'][200000])

In [8]:
# Look at data
metadata.head(10)

,datetime,image_path,audio_path,ln_ufp_num_10s_ma_image_label_raw,ln_ufp_num_10s_ma_spec_label_raw,ln_ufp_size_10s_ma_image_label_raw,ln_ufp_size_10s_ma_spec_label_raw,ln_noise_10s_ma_image_label_raw,ln_noise_10s_ma_spec_label_raw,ufp_num_10s_ma_image_label_raw,...,ufp_size_10s_ma_spec_label_quartile,ufp_size_10s_ma_image_label_quartile,image_extension,audio_extension,image_name,audio_name,pair_pm25,set_V3ext,set,vsby_nm
0,2019-04-23T10:20:27Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.744668,9.744668,3.098740,3.098740,4.319752,4.319752,17063.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
1,2019-04-23T10:20:28Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.710145,9.710145,3.119276,3.119276,4.309322,4.309322,16484.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
2,2019-04-23T10:20:29Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.683153,9.683153,3.144583,3.144583,4.327702,4.327702,16045.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
3,2019-04-23T10:20:30Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.583558,9.583558,3.167583,3.167583,4.320018,4.320018,14524.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
4,2019-04-23T10:20:31Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.509259,9.509259,3.184698,3.184698,4.303119,4.303119,13484.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
5,2019-04-23T10:20:32Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.479069,9.479069,3.197448,3.197448,4.277360,4.277360,13083.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
6,2019-04-23T10:20:33Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.422544,9.422544,3.211650,3.211650,4.273606,4.273606,12364.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
7,2019-04-23T10:20:34Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.399638,9.399638,3.223664,3.223664,4.260000,4.260000,12084.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
8,2019-04-23T10:20:35Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.403107,9.403107,3.235536,3.235536,4.227709,4.227709,12126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0
9,2019-04-23T10:20:36Z,archived files no longer used/GoPro Model/data...,archived files no longer used/GoPro Model/data...,9.406976,9.406976,3.241029,3.241029,4.200804,4.200804,12173.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trn,trn,30.0


In [9]:
# How much data in trn, val, tst sets for metadata_random_split

metadata.set.value_counts()

set
trn    345680
val     22407
tst     22223
Name: count, dtype: int64

In [10]:
#Select input file type (this tells python where to look for the file paths)
file = input("What input are you using: images, or spectrograms? ")

if file == 'images':
    file = 'image_path'
else:
  if file == 'spectrograms':
      file = 'audio_path'
  else:
    print('!!!TYPO in input_data name')

What input are you using: images, or spectrograms?  spectrograms


In [11]:
# Define Initial Learning Rate
initial_learning_rate = 1e-4
meta_data_name = "gp_v2_v3_Training_Only"

In [12]:
# Model architectures
architecture = input("What architecture do you want to use: ConvNeXT_linear_AdamW_Base, ConvNeXT_linear_AdamW_Large, ConvNeXt_Tiny_linear_AdamW, ResNet50_AdamW_Nadam ?")
from tensorflow.keras import backend as Kb

def custom_mse(y_true, y_pred):
    loss = Kb.mean(Kb.square(y_true - y_pred), axis=-1)
    tf.print("y_true:", y_true)
    tf.print("y_pred:", y_pred)
    tf.print("loss:", loss)
    return loss
    
if architecture == 'ConvNeXT_linear_AdamW_Base':
    architecture_preprocessing = K.applications.convnext.preprocess_input
    def get_compiled_model():
        model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
        conv_base = K.applications.ConvNeXtBase(model_name="convnext_base",include_top=False, weights="imagenet", input_tensor=model_input)
        model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
        model_output = K.layers.Dense(units=1, activation='linear')(model_output) 
        model = K.models.Model(inputs=model_input, outputs=model_output)
        model.compile(
            optimizer=K.optimizers.AdamW(learning_rate=initial_learning_rate),
            loss='mse',
            metrics=['mae']
        )
        return model

elif architecture == 'ConvNeXT_linear_AdamW_Large':
    architecture_preprocessing = K.applications.convnext.preprocess_input
    def get_compiled_model():
        model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
        conv_base = K.applications.ConvNeXtLarge(include_top=False, weights="imagenet", input_tensor=model_input)
        model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
        model_output = K.layers.Dense(units=1, activation='linear')(model_output)
        model = K.models.Model(inputs=model_input, outputs=model_output)
        model.compile(
            optimizer=K.optimizers.AdamW(learning_rate=initial_learning_rate),
            loss='mse',
            metrics=['mae']
        )
        return model

elif architecture == 'ConvNeXt_Tiny_AdamW_Nadam':
    architecture_preprocessing = K.applications.convnext.preprocess_input

    def get_compiled_model():
        model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
        conv_base = K.applications.ConvNeXtTiny(include_top=False, weights="imagenet", input_tensor=model_input)
        model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
        model_output = K.layers.Dense(units=1, activation='linear')(model_output)
    
        # Learning rate schedule
        initial_learning_rate = 1e-5
        lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=initial_learning_rate,
            decay_steps=10000,
            alpha=0.0
        )
        optimizer = K.optimizers.AdamW(learning_rate=lr_schedule, clipnorm=1.0)
        model = K.models.Model(inputs=model_input, outputs=model_output)
        model.compile(
            optimizer=optimizer,
            loss='mean_squared_error',
            metrics=['mae']
        )
        return model

elif architecture == 'ResNet50_AdamW_Nadam':
    architecture_preprocessing = K.applications.resnet50.preprocess_input
    def get_compiled_model():
        model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
        conv_base = K.applications.ResNet50(include_top=False, weights= "imagenet", input_tensor=model_input)
        model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
        model_output = K.layers.Dense(units= 1, activation='linear')(model_output) 
        model = K.models.Model(inputs=model_input, outputs=model_output)
        model.compile(
            optimizer=K.optimizers.AdamW(learning_rate = initial_learning_rate),
            loss = custom_mse,
            metrics = ['mae']
        )
        return model
                        
else:
    print('!!!TYPO in architecture name')

What architecture do you want to use: ConvNeXT_linear_AdamW_Base, ConvNeXT_linear_AdamW_Large, ConvNeXt_Tiny_linear_AdamW, ResNet50_AdamW_Nadam ? ConvNeXT_linear_AdamW_Base


In [13]:
# Select Exposure to be modelled
cwd = os.getcwd()
print("Current working directory:", cwd)

target = input("What do you want to model: ln_noise_10s_ma_spec_label_raw, ln_noise_10s_ma_image_label_raw, ln_ufp_num_10s_ma_spec_label_raw, ln_ufp_num_10s_ma_image_label_raw, ln_ufp_size_10s_ma_image_label_raw, ln_ufp_size_10s_ma_spec_label_raw ?")                  

if target == 'ln_noise_10s_ma_image_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # file = image_path or audio_path
                                                y_col= target,  # taget = exposure to be modelled
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

elif target == 'ln_noise_10s_ma_spec_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
elif target == 'ln_ufp_num_10s_ma_image_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    
    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

elif target == 'ln_ufp_num_10s_ma_spec_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
elif target == 'ln_ufp_size_10s_ma_image_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)
 
    
    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
elif target == 'ln_ufp_size_10s_ma_spec_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
else:
    print('!!!TYPO in exposure name') 

Current working directory: /home/scott/Dropbox/IMAGINE Project/MSSI_Project


What do you want to model: ln_noise_10s_ma_spec_label_raw, ln_noise_10s_ma_image_label_raw, ln_ufp_num_10s_ma_spec_label_raw, ln_ufp_num_10s_ma_image_label_raw, ln_ufp_size_10s_ma_image_label_raw, ln_ufp_size_10s_ma_spec_label_raw ? ln_noise_10s_ma_spec_label_raw


Found 345680 validated image filenames.
Found 22407 validated image filenames.


In [13]:
# # Retrieve a batch from the train generator
# x_batch, y_batch = next(train_generator)

# # Print the shape of the batch
# print(f"x_batch shape: {x_batch.shape}, y_batch shape: {y_batch.shape}")

# # Print and visualize some samples from the batch
# num_samples_to_display = 5  # Number of samples to display

# for i in range(num_samples_to_display):
#     plt.imshow(x_batch[i].astype('uint8'))  # Convert to uint8 if necessary
#     plt.title(f"Label: {y_batch[i]}")
#     plt.axis('off')
#     plt.show()

In [15]:
# Retrieve a batch from the validate generator
# x_val_batch, y_val_batch = next(validate_generator)

# # Print the shape of the batch
# print(f"x_val_batch shape: {x_val_batch.shape}, y_val_batch shape: {y_val_batch.shape}")

# # Print and visualize some samples from the batch
# num_samples_to_display = 5  # Number of samples to display

# for i in range(num_samples_to_display):
#     plt.imshow(x_val_batch[i].astype('uint8'))  # Convert to uint8 if necessary
#     plt.title(f"Label: {y_val_batch[i]}")
#     plt.axis('off')
#     plt.show()

In [12]:
# Check a batch from the train generator
# x_batch, y_batch = next(train_generator)
# print(np.isnan(x_batch).any(), np.isnan(y_batch).any())

# # Check a batch from the validate generator
# x_batch, y_batch = next(validate_generator)
# print(np.isnan(x_batch).any(), np.isnan(y_batch).any())

False False
False False


In [ ]:
#Train the model

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

# Open a strategy scope.
with strategy.scope():
    model = get_compiled_model()
    

#Fit Model
model.fit(train_generator,
          validation_data=validate_generator,
          epochs=10,
          steps_per_epoch=int(np.ceil(train_generator.samples/train_generator.batch_size)),
          validation_steps=int(np.ceil(validate_generator.samples/validate_generator.batch_size)),
          callbacks=[csv_logger, 
                     reduce_lr_on_plateau, 
                     model_checkpoint,
                     early_stopping] 
         )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2024-06-12 15:39:55.958979: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11162 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:05:00.0, compute capability: 6.1
2024-06-12 15:39:55.959550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 11409 MB memory:  -> device: 1, name: NVIDIA TITAN Xp, pci bus id: 0000:06:00.0, compute capability: 6.1
2024-06-12 15:39:55.960026: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 11409 MB memory:  -> device: 2, name: NVIDIA TITAN Xp, pci bus id: 0000:09:00.0, compute capability: 6.1
2024-06-12 15:39:55.960492: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 11410 MB memory:  -> device: 3, name: NVIDIA TITAN Xp, pci bus id: 0000:0a:00.0, comp

Epoch 1/10
INFO:tensorflow:Collective all_reduce tensors: 344 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Collective all_reduce tensors: 344 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost

2024-06-12 15:43:41.498666: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-06-12 15:43:42.221571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-06-12 15:43:42.407592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-06-12 15:43:42.756309: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-06-12 15:43:43.448315: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f33a7437670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-12 15:43:43.448348: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN Xp, Compute Capability 6.1
2024-06-12 15:43:43.448356: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA TITAN Xp, Compute Capability 6.1
2024-06-12 15:43:43.448363: I tensorflow/

5402/5402 [==============================] - ETA: 0s - loss: 0.0123 - mae: 0.0690INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Assets written to: ./rishabh_model_files/models/ln_noise_10s_ma_spec_label_raw,ConvNeXT_linear_AdamW_Base,audio_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf/assets


...

5402/5402 [==============================] - 4971s 920ms/step - loss: 0.0036 - mae: 0.0444 - val_loss: 0.0041 - val_mae: 0.0452 - lr: 1.0000e-04
Epoch 4/10
3097/5402 [================>.............] - ETA: 33:33 - loss: 0.0027 - mae: 0.0380

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



5385/5402 [============================>.] - ETA: 14s - loss: 0.0027 - mae: 0.0373INFO:tensorflow:Assets written to: ./rishabh_model_files/models/ln_noise_10s_ma_spec_label_raw,ConvNeXT_linear_AdamW_Base,audio_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf/assets


INFO:tensorflow:Assets written to: ./rishabh_model_files/models/ln_noise_10s_ma_spec_label_raw,ConvNeXT_linear_AdamW_Base,audio_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf/assets


5402/5402 [==============================] - 4973s 920ms/step - loss: 0.0027 - mae: 0.0373 - val_loss: 0.0041 - val_mae: 0.0456 - lr: 1.0000e-04
Epoch 5/10
2869/5402 [==============>...............] - ETA: 36:48 - loss: 0.0022 - mae: 0.0334

In [16]:
#Identify Best Epoch
res = pd.read_csv(filepath_or_buffer='./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')
res = res.sort_values('val_loss', ascending=True).reset_index(drop=True)
best_epoch = res.epoch[0]
best_val_loss = res.val_loss[0]
best_val_mae = res.val_mae[0]
print("best_epoch:", best_epoch)
print("best_val_loss:", best_val_loss)
print("best_val_mae:", best_val_mae)

best_epoch: 6
best_val_loss: 0.0040341755375266
best_val_mae: 0.0478566065430641


In [14]:
# # test code with ResNet50 and CIFAR10 dataset
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers, models
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Load CIFAR-10 dataset
# metad = keras.datasets.cifar10.load_data()
# (x_train, y_train), (x_test, y_test) = metad
# print(x_train)

# # Normalize the data
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0

# # Convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, 10)
# y_test = keras.utils.to_categorical(y_test, 10)

# # Data augmentation
# datagen = ImageDataGenerator(
#     featurewise_center=False,
#     samplewise_center=False,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=False,
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=False
# )
# datagen.fit(x_train)

# def create_resnet50_model():
#     model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
#     conv_base = K.applications.ResNet50(include_top=False, weights= "imagenet", input_tensor=model_input)
#     model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
#     model_output = K.layers.Dense(units= 1, activation='linear')(model_output) 
#     model = K.models.Model(inputs=model_input, outputs=model_output)
#     model.compile(
#         optimizer=K.optimizers.Nadam(learning_rate = initial_learning_rate),
#         loss = 'mse',
#         metrics = ['mae']
#     )
#     return model

# # Create the model
# model = create_resnet50_model()

# # Define callbacks
# callbacks = [
#     keras.callbacks.ModelCheckpoint("resnet50_cifar10.h5", save_best_only=True),
#     keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1),
#     keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# ]

# # Train the model
# model.fit(datagen.flow(x_train, y_train, batch_size=64),
#           validation_data=(x_test, y_test),
#           epochs=50,
#           callbacks=callbacks)

# # Evaluate the model
# test_loss, test_acc = model.evaluate(x_test, y_test)
# print(f"Test accuracy: {test_acc}")

In [15]:
# # experiment with no data augmentation
# # experimented with 'relu' activation
# # experimented with loer learning rae (1e-7)
# from tensorflow.keras import backend as Kb

# architecture = 'ResNet50_linear_Nadam'
# target = 'ln_noise_10s_ma_image_label_raw'
# initial_learning_rate = 1e-4
# meta_data_name = "gp_v2_v3_Training_Only"

# def custom_mse(y_true, y_pred):
#     loss = Kb.mean(Kb.square(y_true - y_pred), axis=-1)
#     tf.print("y_true:", y_true)
#     tf.print("y_pred:", y_pred)
#     tf.print("loss:", loss)
#     return loss

# # architecture_preprocessing = K.applications.resnet50.preprocess_input
# architecture_preprocessing = tf.keras.applications.resnet.preprocess_input

# # Initialize ImageDataGenerator without augmentation
# generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing)

# # Create training and validation generators
# train_generator = generator.flow_from_dataframe(
#     dataframe=metadata.loc[metadata['set'] == 'trn', [target, file]].reset_index(drop=True),
#     x_col=file,  # file = image_path or audio_path
#     y_col=target,  # target = exposure to be modeled
#     class_mode='raw',
#     target_size=(256, 256),  # all of our images will be resized to 256 x 256
#     color_mode='rgb',
#     batch_size=64,
#     shuffle=True
# )

# validate_generator = generator.flow_from_dataframe(
#     dataframe=metadata.loc[metadata['set'] == 'val', [target, file]].reset_index(drop=True),
#     x_col=file,
#     y_col=target,
#     class_mode='raw',
#     target_size=(256, 256),
#     color_mode='rgb',
#     batch_size=64,
#     shuffle=False
# )

# # Define callbacks
# csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/' + target + ',' + architecture + ',' + file + ',' + meta_data_name + '_Combined_IMAGINE_goproUFP_V2_V3_04252024.csv')
# model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/' + target + ',' + architecture + ',' + file + ',' + meta_data_name + '_Combined_IMAGINE_goproUFP_V2_V3_04252024.tf', monitor='val_loss', mode='auto', save_weights_only=False, save_best_only=True)
# reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
# early_stopping = K.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


# def get_compiled_model():
#     model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
#     conv_base = K.applications.ResNet50(include_top=False, weights= "imagenet", input_tensor=model_input)
#     model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
#     # model_output = K.layers.BatchNormalization()(model_output)  # Add BatchNormalization layer
#     model_output = K.layers.Dense(units= 1, activation='linear')(model_output) 
#     model = K.models.Model(inputs=model_input, outputs=model_output)
#     model.compile(
#         optimizer=K.optimizers.Nadam(learning_rate = initial_learning_rate),
#         loss = 'mse',
#         metrics = ['mae']
#     )
#     return model

# # Create a MirroredStrategy for distributed training
# strategy = tf.distribute.MirroredStrategy()

# # Open a strategy scope and compile the model
# with strategy.scope():
#     model = get_compiled_model()

# # Check for NaNs in input data batches
# x_batch, y_batch = next(train_generator)
# print("NaNs in training batch:", np.isnan(x_batch).any(), np.isnan(y_batch).any())

# x_val_batch, y_val_batch = next(validate_generator)
# print("NaNs in validation batch:", np.isnan(x_val_batch).any(), np.isnan(y_val_batch).any())

# # Train the model
# model.fit(
#     train_generator,
#     validation_data=validate_generator,
#     epochs=10,
#     callbacks=[csv_logger, reduce_lr_on_plateau, model_checkpoint, early_stopping]
# )
